In [208]:
# Proceso de datos - ETL 01
# Script para la descarga y lectura de los datos.

import pandas as pd
from pathlib import Path

# Directorio de datos raw 
RAW = Path("data/raw")

#Directorio de datos procesados
PROCESSED = Path("data/processed")

#Leer los archivos CSV
sales = pd.read_csv(RAW / "sales_train.csv")
test  = pd.read_csv(RAW / "test.csv")
items = pd.read_csv(RAW / "items_en.csv")
shops = pd.read_csv(RAW / "shops_en.csv")
cats  = pd.read_csv(RAW / "item_categories_en.csv")
sample  = pd.read_csv(RAW / "sample_submission.csv")

print("sales:", sales.shape)
print("test :", test.shape)
print("items:", items.shape)
print("shops:", shops.shape)
print("cats :", cats.shape)
print("sample :", sample.shape)

test.head()
items.head()
shops.head()
cats.head(20)
sample.head()
sales.head()

sales: (2935849, 6)
test : (214200, 3)
items: (22170, 3)
shops: (60, 2)
cats : (84, 2)
sample : (214200, 2)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [209]:
# Generamos la tabla principal para el proceso de datos
# unimos las tablas relevantes:

df = (
    sales
    .merge(items, on="item_id", how="left")
    .merge(shops, on="shop_id", how="left")
    .merge(cats, on="item_category_id", how="left")
)

print("df :", df.shape)
df.head()


df : (2935849, 10)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,shop_name,item_category_name
0,02.01.2013,0,59,22154,999.00,1.0,Scene 2012 (BD),37,"Yaroslavl shopping center ""Altair""",Movies - Blu-Ray
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,"Moscow TRC ""Atrium""",Music - Vinyl
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,"Moscow TRC ""Atrium""",Music - Vinyl
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,"Moscow TRC ""Atrium""",Music - Vinyl
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,"Moscow TRC ""Atrium""",Music - CD production firm


In [211]:
# Transformaciones y limpieza de datos

df["item_price"] = df["item_price"].astype(float)
df["item_cnt_day"] = pd.to_numeric(df["item_cnt_day"])

# Ajustamos tipo de datos de fecha
df["date"] = pd.to_datetime(df["date"], format="%d.%m.%Y")

# calculamos ventas diarias (Sales) cantidas * precio
df["sales"] = (df["item_cnt_day"] * df["item_price"]).astype(float)

#Validados tipo de datos numericos
df["sales"] = df["sales"].astype(float)

# Validamos el dataframe
print("df :", df.shape)
df.head(5)


df : (2935849, 11)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,shop_name,item_category_name,sales
0,2013-01-02,0,59,22154,999.00,1.0,Scene 2012 (BD),37,"Yaroslavl shopping center ""Altair""",Movies - Blu-Ray,999.00
1,2013-01-03,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,"Moscow TRC ""Atrium""",Music - Vinyl,899.00
2,2013-01-05,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,"Moscow TRC ""Atrium""",Music - Vinyl,-899.00
3,2013-01-06,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,"Moscow TRC ""Atrium""",Music - Vinyl,1709.05
4,2013-01-15,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,"Moscow TRC ""Atrium""",Music - CD production firm,1099.00


In [212]:
#Cifras control de las ventas totales en formato amigable

# Asegurar fecha
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Año
df["year"] = df["date"].dt.year

yearly_control = (
    df
    .groupby("year", as_index=False)
    .agg(
        total_sales=("sales", "sum"),
        total_units=("item_cnt_day", "sum"),
        num_transactions=("item_cnt_day", "size"),
        avg_price=("item_price", "mean"),
        active_products=("item_id", "nunique"),
        active_shops=("shop_id", "nunique"),
    )
)
yearly_control_fmt = yearly_control.copy()

yearly_control_fmt["total_sales"] = yearly_control_fmt["total_sales"].apply(
    lambda x: f"${x:,.2f}"
)

yearly_control_fmt["total_units"] = yearly_control_fmt["total_units"].apply(
    lambda x: f"{int(x):,}"
)

yearly_control_fmt["num_transactions"] = yearly_control_fmt["num_transactions"].apply(
    lambda x: f"{int(x):,}"
)

yearly_control_fmt["avg_price"] = yearly_control_fmt["avg_price"].apply(
    lambda x: f"${x:,.2f}"
)

yearly_control_fmt["active_products"] = yearly_control_fmt["active_products"].apply(
    lambda x: f"{int(x):,}"
)

yearly_control_fmt["active_shops"] = yearly_control_fmt["active_shops"].apply(
    lambda x: f"{int(x):,}"
)

yearly_control_fmt


,year,total_sales,total_units,num_transactions,avg_price,active_products,active_shops
0,2013,"$1,217,524,733.98","1,562,733","1,267,562",$726.80,"14,971",52
1,2014,"$1,346,778,478.81","1,320,889","1,055,861",$988.34,"14,108",53
2,2015,"$834,623,131.78","764,584","612,426","$1,062.32","11,249",54


In [213]:
# Genramos primera salida para realizar el EDA principal en formato csv

df.to_csv(PROCESSED / "df_base.csv", index=False)


In [221]:
# Agregación mensual: tienda + producto 

# 1) Asegurar fecha en datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# 2) Agregación mensual (ME = month-end)
monthly = (
    df
    .groupby(
        [pd.Grouper(key="date", freq="ME"), "shop_id", 
         "item_id", "item_name", ],
        as_index=False
    )
    .agg(
        monthly_sales=("sales", "sum"),
        monthly_units=("item_cnt_day", "sum"),
        avg_price=("item_price", "mean"),
        min_price=("item_price", "min"),    
        max_price=("item_price", "max"),
        num_transactions=("item_cnt_day", "size"),
        active_days=("date", lambda s: s.dt.date.nunique()),
    )
)

# 3) Agregar año y mes
monthly["year"] = monthly["date"].dt.year
monthly["month"] = monthly["date"].dt.month

monthly.head()

,date,shop_id,item_id,item_name,monthly_sales,monthly_units,avg_price,min_price,max_price,num_transactions,active_days,year,month
0,2013-01-31,0,32,1+1,1326.0,6.0,221.0,221.0,221.0,4,4,2013,1
1,2013-01-31,0,33,1+1 (BD),1041.0,3.0,347.0,347.0,347.0,3,3,2013,1
2,2013-01-31,0,35,10 YEARS LATER,247.0,1.0,247.0,247.0,247.0,1,1,2013,1
3,2013-01-31,0,43,EUR 100 million,221.0,1.0,221.0,221.0,221.0,1,1,2013,1
4,2013-01-31,0,51,100 best classical works (mp3-CD) (Digipack),257.0,2.0,128.5,127.0,130.0,2,2,2013,1


In [230]:
# Validamos datos y salidas

print("Filas originales:", df.shape[0])
print("Filas mensuales:", monthly.shape[0])

print("monthly :", monthly.shape)
monthly_fmt.head()

Filas originales: 2935849
Filas mensuales: 1609124
monthly : (1609124, 13)


,date,shop_id,item_id,item_name,item_category_id,monthly_sales,monthly_units,avg_price,min_price,max_price,num_transactions,active_days,year,month
0,2013-01-31,0,32,1+1,40,"$1,326.00",6,$221.00,$221.00,$221.00,4,4,2013,1
1,2013-01-31,0,33,1+1 (BD),37,"$1,041.00",3,$347.00,$347.00,$347.00,3,3,2013,1
2,2013-01-31,0,35,10 YEARS LATER,40,$247.00,1,$247.00,$247.00,$247.00,1,1,2013,1
3,2013-01-31,0,43,EUR 100 million,40,$221.00,1,$221.00,$221.00,$221.00,1,1,2013,1
4,2013-01-31,0,51,100 best classical works (mp3-CD) (Digipack),57,$257.00,2,$128.50,$127.00,$130.00,2,2,2013,1


In [231]:
# Validamos datos  en formato amigable
#cifras de control anuales

annual = (
    monthly
    .groupby(
        ["year"],
        as_index=False
    )
    .agg(
        annual_sales=("monthly_sales", "sum"),
        annual_units=("monthly_units", "sum"),
        avg_price=("avg_price", "mean"),
        min_price=("min_price", "min"),
        max_price=("max_price", "max"),
        num_transactions=("num_transactions", "sum"),
        active_days=("active_days", "sum")
    )
)
# Formato amigable
# Copia solo para presentación
annual_fmt = annual.copy()

# Formato dólares
annual_fmt["annual_sales"] = annual_fmt["annual_sales"].apply(
    lambda x: f"${x:,.2f}"
)

annual_fmt["avg_price"] = annual_fmt["avg_price"].apply(
    lambda x: f"${x:,.2f}"
)

annual_fmt["min_price"] = annual_fmt["min_price"].apply(
    lambda x: f"${x:,.2f}"
)

annual_fmt["max_price"] = annual_fmt["max_price"].apply(
    lambda x: f"${x:,.2f}"
)

# Formato miles (enteros)
annual_fmt["annual_units"] = annual_fmt["annual_units"].apply(
    lambda x: f"{int(x):,}"
)

annual_fmt["num_transactions"] = annual_fmt["num_transactions"].apply(
    lambda x: f"{int(x):,}"
)

annual_fmt["active_days"] = annual_fmt["active_days"].apply(
    lambda x: f"{int(x):,}"
)

# (Opcional) Renombrar columnas para lectura ejecutiva
annual_fmt = annual_fmt.rename(columns={
    "year": "Año",
    "annual_sales": "Ventas anuales ($)",
    "annual_units": "Unidades anuales",
    "avg_price": "Precio promedio ($)",
    "min_price": "Precio mínimo ($)",
    "max_price": "Precio máximo ($)",
    "num_transactions": "Transacciones",
    "active_days": "Días activos"
})

annual_fmt

annual_fmt.head()

,Año,Ventas anuales ($),Unidades anuales,Precio promedio ($),Precio mínimo ($),Precio máximo ($),Transacciones,Días activos
0,2013,"$1,217,524,733.98","1,562,733",$657.96,$-1.00,"$307,980.00","1,267,562","1,267,551"
1,2014,"$1,346,778,478.81","1,320,889",$827.93,$0.50,"$50,999.00","1,055,861","1,055,848"
2,2015,"$834,623,131.78","764,584",$989.81,$0.91,"$49,782.00","612,426","612,422"


In [232]:
# Generamos lags para ventas y unidades mensuales

# Agregación mensual: tienda, producto (con TODAS las variables) y lags

# 1) Asegurar fecha en datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# 2) Asegurar sales (si no existe o si quieres recalcularla)
if "sales" not in df.columns:
    df["sales"] = (df["item_cnt_day"] * df["item_price"]).astype(float)
else:
    df["sales"] = df["sales"].astype(float)

# 3) Agregación mensual (ME = month-end)
monthly = (
    df
    .groupby([pd.Grouper(key="date", freq="ME"), "shop_id", "item_id", "item_name"], as_index=False)
    .agg(
        monthly_sales=("sales", "sum"),
        monthly_units=("item_cnt_day", "sum"),
        avg_price=("item_price", "mean"),
        min_price=("item_price", "min"),
        max_price=("item_price", "max"),
        num_transactions=("item_cnt_day", "size"),
        active_days=("date", lambda s: s.dt.date.nunique()),
    )
)

# 4) Agregar año y mes
monthly["year"] = monthly["date"].dt.year
monthly["month"] = monthly["date"].dt.month

# 5) Ordenar (clave para lags)
monthly = monthly.sort_values(by=["shop_id", "item_id", "year", "month"]).reset_index(drop=True)

# 6) Lags (monto y unidades) para el mismo producto y tienda
monthly["monthly_sales_lag_1"] = (
    monthly.groupby(["shop_id", "item_id"])["monthly_sales"].shift(1)
)
monthly["monthly_units_lag_1"] = (
    monthly.groupby(["shop_id", "item_id"])["monthly_units"].shift(1)
)

# 7) Rellenar NaN solo en lags (primer mes de cada serie)
monthly[["monthly_sales_lag_1", "monthly_units_lag_1"]] = (
    monthly[["monthly_sales_lag_1", "monthly_units_lag_1"]].fillna(0)
)

monthly.head()

,date,shop_id,item_id,item_name,monthly_sales,monthly_units,avg_price,min_price,max_price,num_transactions,active_days,year,month,monthly_sales_lag_1,monthly_units_lag_1
0,2013-02-28,0,30,"007: COORDINATES ""SKAYFOLL""",8215.0,31.0,265.0,265.0,265.0,9,9,2013,2,0.0,0.0
1,2013-02-28,0,31,"007: COORDINATES ""SKAYFOLL» (BD)",4774.0,11.0,434.0,434.0,434.0,7,7,2013,2,0.0,0.0
2,2013-01-31,0,32,1+1,1326.0,6.0,221.0,221.0,221.0,4,4,2013,1,0.0,0.0
3,2013-02-28,0,32,1+1,2210.0,10.0,221.0,221.0,221.0,7,7,2013,2,1326.0,6.0
4,2013-01-31,0,33,1+1 (BD),1041.0,3.0,347.0,347.0,347.0,3,3,2013,1,0.0,0.0


In [237]:
monthly.tail(90)

,date,shop_id,item_id,item_name,monthly_sales,monthly_units,avg_price,min_price,max_price,num_transactions,active_days,year,month,monthly_sales_lag_1,monthly_units_lag_1
1609034,2013-08-31,59,22091,BATTERIES DURACELL TURBO LR March 2 * BL,218.0,2.0,109.0,109.0,109.0,2,2,2013,8,763.0,7.0
1609035,2013-11-30,59,22091,BATTERIES DURACELL TURBO LR March 2 * BL,109.0,1.0,109.0,109.0,109.0,1,1,2013,11,218.0,2.0
1609036,2013-12-31,59,22091,BATTERIES DURACELL TURBO LR March 2 * BL,109.0,1.0,109.0,109.0,109.0,1,1,2013,12,109.0,1.0
1609037,2014-01-31,59,22091,BATTERIES DURACELL TURBO LR March 2 * BL,109.0,1.0,109.0,109.0,109.0,1,1,2014,1,109.0,1.0
1609038,2014-06-30,59,22091,BATTERIES DURACELL TURBO LR March 2 * BL,109.0,1.0,109.0,109.0,109.0,1,1,2014,6,109.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609119,2015-04-30,59,22164,FURY (BD),1398.0,2.0,699.0,699.0,699.0,2,2,2015,4,749.0,1.0
1609120,2015-07-31,59,22164,FURY (BD),699.0,1.0,699.0,699.0,699.0,1,1,2015,7,1398.0,2.0
1609121,2013-10-31,59,22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,299.0,1.0,299.0,299.0,299.0,1,1,2013,10,0.0,0.0
1609122,2013-12-31,59,22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,598.0,2.0,299.0,299.0,299.0,2,2,2013,12,299.0,1.0


In [238]:
# Guardamos el dataset mensual con lag en formato csv

monthly.to_csv(PROCESSED / "monthly_with_lags.csv", index=False)